In [122]:
!pip install numpy
!pip install gym
import numpy as np
import gym
import random
import time
from IPython.display import clear_output
from gym.envs.toy_text.frozen_lake import generate_random_map


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: C:\Users\clayb\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: C:\Users\clayb\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [123]:
env = gym.make('FrozenLake-v1', render_mode='ansi')

In [124]:
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size, action_space_size))

print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [125]:
def load_frozen_lake(desc=None, map_name=None, is_slippery=False):
    """
    Load the FrozenLake environment from OpenAI's gym.

    Parameters:
    desc -- None or a list of lists containing a custom
    description of the map to load for the environment
    map_name -- None or a string containing the pre-made map to load
    is_slippery -- a boolean to determine if the ice is slippery

    If both desc and map_name are None, the environment will
    load a randomly generated 8x8 map.

    Returns:
    The FrozenLake environment.
    """
    if desc is None and map_name is None:
        desc = generate_random_map(size=8)

    return gym.make('FrozenLake-v1', desc=desc,
                    map_name=map_name, is_slippery=is_slippery)

In [154]:
np.random.seed(0)
env = load_frozen_lake()
print(env.desc)
print(env.P[0][0])
env = load_frozen_lake(is_slippery=True)
print(env.desc)
print(env.P[0][0])
desc = [['S', 'F', 'F'], ['F', 'H', 'H'], ['F', 'F', 'G']]
env = load_frozen_lake(desc=desc)
print(env.desc)
env = load_frozen_lake(map_name='4x4')
print(env.desc)

[[b'S' b'F' b'F' b'F' b'F' b'F' b'F' b'H']
 [b'H' b'F' b'F' b'F' b'F' b'H' b'F' b'F']
 [b'F' b'H' b'F' b'H' b'H' b'F' b'F' b'F']
 [b'F' b'F' b'F' b'H' b'F' b'F' b'F' b'F']
 [b'F' b'F' b'F' b'F' b'F' b'F' b'H' b'F']
 [b'F' b'F' b'F' b'F' b'F' b'F' b'F' b'F']
 [b'F' b'F' b'F' b'F' b'H' b'F' b'F' b'F']
 [b'F' b'F' b'F' b'F' b'F' b'F' b'F' b'G']]
[(1.0, 0, 0.0, False)]
[[b'S' b'F' b'H' b'F' b'H' b'F' b'H' b'F']
 [b'H' b'F' b'F' b'F' b'F' b'F' b'F' b'F']
 [b'F' b'F' b'F' b'F' b'F' b'F' b'F' b'F']
 [b'F' b'H' b'F' b'F' b'F' b'F' b'F' b'F']
 [b'F' b'F' b'H' b'F' b'F' b'F' b'F' b'H']
 [b'F' b'F' b'F' b'F' b'F' b'H' b'F' b'H']
 [b'F' b'F' b'H' b'F' b'H' b'F' b'H' b'F']
 [b'F' b'F' b'H' b'F' b'F' b'F' b'F' b'G']]
[(0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 8, 0.0, True)]
[[b'S' b'F' b'F']
 [b'F' b'H' b'H']
 [b'F' b'F' b'G']]
[[b'S' b'F' b'F' b'F']
 [b'F' b'H' b'F' b'H']
 [b'F' b'F' b'F' b'H']
 [b'H' b'F' b'F' b'G']]


In [156]:
def q_init(env):
    """Initialize Q-table to zeros."""
    return np.zeros([env.observation_space.n, env.action_space.n])

In [157]:
env = load_frozen_lake()
Q = q_init(env)
print(Q.shape)
env = load_frozen_lake(is_slippery=True)
Q = q_init(env)
print(Q.shape)
desc = [['S', 'F', 'F'], ['F', 'H', 'H'], ['F', 'F', 'G']]
env = load_frozen_lake(desc=desc)
Q = q_init(env)
print(Q.shape)
env = load_frozen_lake(map_name='4x4')
Q = q_init(env)
print(Q.shape)

(64, 4)
(64, 4)
(9, 4)
(16, 4)


In [162]:
def epsilon_greedy(Q, state, epsilon):
    """Epsilon-greedy policy."""
    if random.uniform(0, 1) < epsilon:
        action = env.action_space.sample()  # Explore action space
    else:
        action = np.argmax(Q[state])  # Exploit learned values
    return action

In [163]:

desc = [['S', 'F', 'F'], ['F', 'H', 'H'], ['F', 'F', 'G']]
env = load_frozen_lake(desc=desc)
Q = q_init(env)
Q[7] = np.array([0.5, 0.7, 1, -1])
np.random.seed(0)
print(epsilon_greedy(Q, 7, 0.5))
np.random.seed(1)
print(epsilon_greedy(Q, 7, 0.5))

2
0


In [144]:
def train(env, Q, episodes=5000, max_steps=100,
          alpha=0.1, gamma=0.99, epsilon=1,
          min_epsilon=0.1, epsilon_decay=0.05):
    """Train agent to learn Q-values."""
    total_rewards = []
    for episode in range(episodes):
        # initialize new episode parameters
        state = env.reset()[0]
        done = False
        rewards = 0

        for step in range(max_steps):
            # exploration/exploitation trade-off
            exploration_rate_threshold = random.uniform(0, 1)
            if exploration_rate_threshold > epsilon:
                action = np.argmax(Q[state,: ])
            else:
                action = env.action_space.sample()
        new_state, reward, done, truncated, info = env.step(action)
        # Update Q-table for Q(s,a)
        Q[state, action] = Q[state, action] * (1 - alpha) + \
        alpha * (reward + gamma * np.max(Q[new_state, :]))

        state = new_state
        rewards_current_episode += reward 

        if done == True:
            break

        # Exploration rate decay
        alpha = min_epsilon + \
    (max_steps - min_epsilon) * np.exp(-epsilon_decay*episode)

        total_rewards.append(rewards)


    return Q, total_rewards

In [151]:
class DummyEnv:
    def __init__(self):
        self.action_space = self.DummyActionSpace()
        
    class DummyActionSpace:
        def sample(self):
            return np.random.choice([0, 1])  # Assuming two actions: 0 and 1

    def reset(self):
        return 0

    def step(self, action):
        return 1, 1, False, {}


# Main Section
if __name__ == "__main__":
    env = DummyEnv()  # Initialize the environment
    Q = np.zeros((2, 2))  # Initialize the Q-table
    
    # Train the agent and get the final Q-values and total rewards
    Q, total_rewards = train(env, Q)
print(Q)

[[100. 100.]
 [100. 100.]]


In [148]:
def play(env, Q, max_steps=100):
    """Play agent with learned Q-values."""
    state = env.reset()
    total_rewards = 0
    for step in range(max_steps):
        action = np.argmax(Q[state])
        new_state, reward, done, info = env.step(action)
        env.render()
        state = new_state
        total_rewards += reward
        if done:
            break
    return total_rewards

In [149]:
# Print updated Q-table
print("\n\n********Q-table********\n")
print(Q)




********Q-table********

[[100. 100.]
 [100. 100.]]
